In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import requests
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
# Load Excel file
excel_file = "/content/drive/MyDrive/Colab Notebooks/loinc_codes.xlsx"

# API Base URL (example, update as needed)
api_url = "https://loinc.regenstrief.org/searchapi/loincs"

# Authentication credentials
auth = ("davissiemens", "jejben-3rykVi-fejzaf")

In [51]:
# Function to get ranking for a LOINC code
def get_loinc_data(loinc_num):
    params = {"query": loinc_num, "rows": 1}  # Search parameter
    response = requests.get(api_url, params=params, auth=auth)

    if response.status_code == 200:
        data = response.json()
        if "Results" in data and len(data["Results"]) > 0:
            # Extract the COMMON_TEST_RANK
            return data["Results"][0].get("COMMON_TEST_RANK", "No Rank Found")
    return "Not Found"

In [57]:
# Function to check if a LOINC code is related to "glucose in blood"
def check_query(loinc_code, query_search):
    # Searching for "glucose in blood" in the LOINC database
    params = {"query": query_search, "rows": 800}  # Adjust query for glucose search
    response = requests.get(api_url, params=params, auth=auth)  # API call with authentication

    if response.status_code == 200:
        data = response.json()
        # Check if any of the results have the same LOINC code
        for result in data.get("Results", []):
            if result.get("LOINC_NUM") == loinc_code:
                return 1
    return 0

In [58]:
# Read all sheets from the Excel file
excel_sheets = pd.read_excel(excel_file, sheet_name=None, skiprows=2)

# Prepare a dictionary to hold the results
result_dict = {}

# Loop through each sheet in the Excel file
for sheet_name, df in excel_sheets.items():
    print(f"Processing sheet: {sheet_name}")
    # Assuming LOINC codes are in a column named 'LOINC Code'
    df["rank"] = df["loinc_num"].astype(str).apply(get_loinc_data)

    df["inSearch"] = df["loinc_num"].astype(str).apply(lambda x: check_query(x, sheet_name))

    # Save the results for each sheet in the result_dict
    result_dict[sheet_name] = df

# Save the results to a new Excel file with multiple sheets
with pd.ExcelWriter("/content/drive/MyDrive/Colab Notebooks/loinc_ranks_query.xlsx") as writer:
    for sheet_name, df in result_dict.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("LOINC rankings saved to loinc_ranks_query.xlsx")

Processing sheet: glucose in blood
Processing sheet: bilirubin in plasma
Processing sheet: White blood cells count
LOINC rankings saved to loinc_ranks_query.xlsx
